<font color=red size=4>构造的思路是取问题中topic实体的其它关系作为负样本</font>

# 导入包

In [1]:
from collections import defaultdict
from tqdm import tqdm
import re
import json
from copy import deepcopy
import random

import torch
import os
import pickle
from transformers import AutoTokenizer
import transformers
import torch.nn as nn
import math
import time
import pandas as pd
import numpy as np
from nlp_basictasks.tasks import Ner,cls
from nlp_basictasks.evaluation import nerEvaluator,clsEvaluator
from nlp_basictasks.readers.cls import InputExample
from collections import Counter
from copy import deepcopy
from nlp_basictasks.modules.transformers import BertTokenizer
device=torch.device('cuda')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# 读取原始数据

In [2]:
def read_data(data_path):
    with open(data_path) as f:
        lines=f.readlines()
    data=[]
    for line in lines:
        data.append(json.loads(line))
    print("原始数据有{}个样本".format(len(data)))
    return data
    
data=[]
data.extend(read_data('/home/xhsun/NLP/KGQA/QApairs/Chinese/kgclue/train.json'))
data.extend(read_data('/home/xhsun/NLP/KGQA/QApairs/Chinese/kgclue/dev.json'))
data.extend(read_data('/home/xhsun/NLP/KGQA/QApairs/Chinese/kgclue/test_public.json'))

原始数据有18000个样本
原始数据有2000个样本
原始数据有2000个样本


In [3]:
print("原始数据有{}个样本".format(len(data)))
for i in range(10):
    print(data[i])

原始数据有22000个样本
{'id': 0, 'question': '你知道守望星光是哪家公司出品的吗？', 'answer': '守望星光（韩玉玲、刘东考演唱歌曲） ||| 出品公司 ||| 韩玉玲原创音乐'}
{'id': 1, 'question': '武汉交通职业学院计算机协会有哪些学校？', 'answer': '武汉交通职业学院计算机协会 ||| 学校 ||| 武汉交通职业学院'}
{'id': 2, 'question': '你知道龙晶用繁体字怎么写吗？', 'answer': '龙晶（游戏） ||| 繁体 ||| 龍晶'}
{'id': 3, 'question': '无罪无我的和声是什么？', 'answer': '无罪无我 ||| 和声 ||| 章韵、徐晨辰'}
{'id': 4, 'question': '你知道影舞者的车身有多重吗？', 'answer': '影舞者（《QQ飞车》中的赛车） ||| 车身重量 ||| 1232kg'}
{'id': 5, 'question': '冯慧专业是什么方向？', 'answer': '冯慧（武汉大学教授） ||| 专业方向 ||| 偏微分方程数值解'}
{'id': 6, 'question': '你知道戴姆勒股份公司有多少员工吗？', 'answer': '戴姆勒股份公司 ||| 员工数 ||| 298655 人(2020年)'}
{'id': 7, 'question': '谁知道魔力w.i.t.c.h.系列有哪些丛书？', 'answer': '魔力w.i.t.c.h. ||| 丛书系列 ||| 魔力w.i.t.c.h.100少女完全手册、魔力w.i.t.c.h.故事集、魔力w.i.t.c.h.小说《破碎的魔球》'}
{'id': 8, 'question': '谁知道端午节有什么重要意义？', 'answer': '端午节（中国四大传统节日之一） ||| 节日意义 ||| 传承与弘扬非物质文化'}
{'id': 9, 'question': '谁唱的风不息？', 'answer': '风不息（1995年郭富城发行的音乐专辑） ||| 歌手 ||| 郭富城'}


# 读取大规模的知识图谱

In [4]:
sub_map = defaultdict(set)
obj_map = defaultdict(set)
so_map=defaultdict(set)
sp_map=defaultdict(set)
alias_dict=defaultdict(set)
kg_path='/home/xhsun/NLP/KGQA/KG/kgCLUE/Knowledge.txt'
with open(kg_path) as f:
    lines=f.readlines()
    
entities_set=set()
relations_set=set()
print(len(lines))

#new_triples=[]
for i in tqdm(range(len(lines))):
    line=lines[i]
    l = line.strip().split('\t')
    s = l[0].strip()
    p = l[1].strip()
    o = l[2].strip()
    if s=='' or p=='' or o=='':
        print('-'*100)
        continue
    entities_set.add(s)
    entities_set.add(o)
    relations_set.add(p)
    #new_triples.append((s,p,o))
    sub_map[s].add((p, o))
    obj_map[o].add((s, p))
    so_map[(s,o)].add(p)
    sp_map[(s,p)].add(o)
    if '（' in s and "）" in s:
        entity_mention=s.split('（')[0]
        alias_dict[entity_mention].add(s)

22883549


100%|███████████████████████████████████████████████████████████████████████████████████████| 22883549/22883549 [02:13<00:00, 171197.04it/s]


In [5]:
print('大规模图谱中一共有{}个三元组，{}个实体，{}个关系'.format(len(lines),len(entities_set),len(relations_set)))

大规模图谱中一共有22883549个三元组，9418104个实体，245837个关系


In [6]:
alias_dict['玻璃']

{'玻璃（2004年上海人民美术出版社出版的图书）',
 '玻璃（克里斯·莱夫特瑞所著图书）',
 '玻璃（北村作品）',
 '玻璃（吴日言演唱歌曲）',
 '玻璃（手机游戏）',
 '玻璃（方糖泡泡演唱歌曲）',
 '玻璃（朱茵演唱的粤语歌曲）',
 '玻璃（游戏《Minecraft》中的方块）',
 '玻璃（玻璃杂志---《玻璃工业》）',
 '玻璃（男同性恋）',
 '玻璃（硅酸盐类非金属材料）',
 '玻璃（红楼梦中的丫鬟名）',
 '玻璃（蔡淳佳演唱的歌曲）',
 '玻璃（辛晓琪的歌曲）',
 '玻璃（陈颖恩演唱歌曲）'}

# 构造关系匹配数据集

这一步需要做两件事：
1. 构造别名字典
2. 构造关系匹配数据

In [22]:
relation_match_data=[]
pos_nums=0
neg_nums=0
for example in tqdm(data):
    id_,question,answer=example['id'],example['question'],example['answer']
    answer_split=answer.split('|||')
    head,relation,tail=answer_split[0].strip(),answer_split[1].strip(),answer_split[2].strip()
    entity_mention=head
    if ('（' in head and '）' in head) and head not in question:
        entity_mention=head.split('（')[0]
        assert entity_mention in alias_dict
    if relation in ['别名','中文名',"英文名","外文名","原名","别称","昵称","全称","中文名称","英文名称"]:
        alias_dict[entity_mention].add(tail)
        
    candidate_pos=sub_map[head]
#     print(relation,candidate_relations)
#     assert relation in candidate_relations
    if entity_mention not in question:
        continue
    question=question.replace(entity_mention,'NE')
    relation_match_data.append({"question":question,"head":head,'relation':relation,'label':'1',"answer":tail})
    pos_nums+=1
    for p,o in candidate_pos:
        if p!=relation:
            relation_match_data.append({"question":question,"head":head,'relation':p,'label':'0','answer':o})               
            neg_nums+=1

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 22000/22000 [00:00<00:00, 179798.38it/s]


In [23]:
print(len(relation_match_data),pos_nums,neg_nums)

201274 21975 179299


In [24]:
relation_match_data[:5]

[{'question': '你知道NE是哪家公司出品的吗？',
  'head': '守望星光（韩玉玲、刘东考演唱歌曲）',
  'relation': '出品公司',
  'label': '1',
  'answer': '韩玉玲原创音乐'},
 {'question': '你知道NE是哪家公司出品的吗？',
  'head': '守望星光（韩玉玲、刘东考演唱歌曲）',
  'relation': '中文名',
  'label': '0',
  'answer': '守望星光'},
 {'question': '你知道NE是哪家公司出品的吗？',
  'head': '守望星光（韩玉玲、刘东考演唱歌曲）',
  'relation': '谱曲',
  'label': '0',
  'answer': '韩玉玲'},
 {'question': '你知道NE是哪家公司出品的吗？',
  'head': '守望星光（韩玉玲、刘东考演唱歌曲）',
  'relation': '歌曲原唱',
  'label': '0',
  'answer': '韩玉玲、刘东考'},
 {'question': '你知道NE是哪家公司出品的吗？',
  'head': '守望星光（韩玉玲、刘东考演唱歌曲）',
  'relation': '歌曲时长',
  'label': '0',
  'answer': '04分58秒'}]

- 针对实体识别的结果，从alias_dict中找出所有的候选实体
- 针对每一个候选实体，从sub_map中找出所有可能的(reltion,tail)
- 将每一个relation与问题模板送入匹配模型中计算每一个reltion的得分（或者将整个三元组与问题模板送入匹配模型中计算整个三元组与问题的得分，选择分数最高的三元组。这种策略可能比单计算关系与问题模板的准确性更高，因为可以利用到消歧的信息和答案的语义信息）

# 保存sub_map、alias_dict、match_data

In [30]:
with open("/home/xhsun/NLP/KGQA/KG/kgCLUE/NER-Match/sub_map.json","w") as f:
    for key,value in sub_map.items():
        sub_map[key]=list(value)
    json.dump(obj=sub_map,fp=f,ensure_ascii=False)
with open("/home/xhsun/NLP/KGQA/KG/kgCLUE/NER-Match/alias_dict.json","w") as f:
    for key,value in alias_dict.items():
        alias_dict[key]=list(value)
    json.dump(obj=alias_dict,fp=f,ensure_ascii=False)
with open("/home/xhsun/NLP/KGQA/KG/kgCLUE/NER-Match/match_data.txt","w") as f:
    for example in relation_match_data:
        f.write(json.dumps(example,ensure_ascii=False)+'\n')

In [27]:
sub_map['中国面向世界']

{('ISBN', '9787108028006'),
 ('书名', '中国面向世界'),
 ('作者', '佘敷华'),
 ('作者', '佘敷华/张芝联'),
 ('出版时间', '1987年6月'),
 ('出版社', '三联书店'),
 ('定价', '2.00'),
 ('定价', '22.5 元'),
 ('统一书号', '2002-317'),
 ('译者', '袁树仁'),
 ('页数', '231页'),
 ('页数', '315 页')}

In [26]:
alias_dict['情人']

{'情人（1990年海峡文艺出版社出版的图书）',
 '情人（2004年上海画报出版社出版的图书）',
 '情人（2004年王裕如编写图书）',
 '情人（2013年广东旅游出版社出版的图书）',
 '情人（2013年朱哲琴与凯撒演唱的歌曲）',
 '情人（Amour饮品）',
 '情人（Beyond演唱歌曲）',
 '情人（上海译文出版社发行书籍）',
 '情人（俄罗斯2002年瓦·托多罗夫斯基执导剧情片）',
 '情人（冰一言演唱歌曲）',
 '情人（刀郎演唱歌曲）',
 '情人（刘威煌演唱歌曲）',
 '情人（巴西2002年亚利桑德罗·阿奎斯提执导电影）',
 '情人（李碧华演唱歌曲）',
 '情人（杜德伟演唱歌曲）',
 '情人（杨林演唱歌曲）',
 '情人（林依轮演唱歌曲）',
 '情人（林忆莲演唱歌曲）',
 '情人（柯云路创作小说）',
 '情人（汉语词语）',
 '情人（法国1992年让·雅克·阿诺执导电影）',
 '情人（潘嘉丽演唱歌曲）',
 '情人（潘美辰演唱歌曲）',
 '情人（玛格丽特·杜拉斯著中篇小说）',
 '情人（苏曼演唱歌曲）',
 '情人（蔡国庆演唱歌曲）',
 '情人（蔡徐坤演唱歌曲）',
 '情人（西奥Sio演唱歌曲）',
 '情人（西班牙2002年胡安·路易斯·伊沃拉执导电影）',
 '情人（许巍演唱歌曲）',
 '情人（谭咏麟演唱歌曲）',
 '情人（谭咏麟音乐专辑）',
 '情人（谷乐著网络小说）',
 '情人（豆瓣阅读作者舍目斯所作的家庭女性小说）',
 '情人（邓紫棋演唱歌曲）',
 '情人（邱海萍执导话剧）',
 '情人（邵明明、贺敬轩演唱歌曲）',
 '情人（阿丽玛演唱歌曲）',
 '情人（陈嘉璐演唱歌曲）',
 '情人（陈百强演唱歌曲）',
 '情人（青岛出版社发行书籍）',
 '情人（韩红演唱歌曲）',
 '情人（黄品源演唱歌曲）'}